In [ ]:
pip install pandas

In [ ]:
import pandas as pd
netflix = pd.read_csv('netflix_titles.csv')
prime = pd.read_csv('amazon_prime_titles.csv')
disney = pd.read_csv('disney_plus_titles.csv')
#Joining all the files together
final_data = pd.concat ([netflix, prime, disney])
final_data.shape

(19925, 15)

In [ ]:
#Keeping only title, release_year, Netflix, Prime and Disney+
data_updated = final_data.drop(['show_id', 'rating', 'director', 'cast', 'country', 'date_added', 'duration', 'listed_in', 'description'], axis=1)
data_updated.head()

,type,title,release_year,Netflix,Prime Video,Disney+
0,0,Dick Johnson Is Dead,2020,1,0,0
1,1,Blood & Water,2021,1,0,0
2,1,Ganglands,2021,1,0,0
3,1,Jailbirds New Orleans,2021,1,0,0
4,1,Kota Factory,2021,1,0,0


In [ ]:
#Reading Moviesonstreamingplatform file
moviesonplatform = pd.read_csv('/content/MoviesOnStreamingPlatforms_updated.csv')
moviesonplatform.shape

(16744, 17)

In [ ]:
#Keeping same columns as in above files
cleaned_moviesonplatform = moviesonplatform.drop(['ID', 'IMDb', 'Unnamed: 0', 'Age', 'Directors', 'Genres', 'Country', 'Language', 'Runtime', 'Rotten Tomatoes'], axis=1)
#Removing all the movies present only on HULU
Hulumovies = cleaned_moviesonplatform[(cleaned_moviesonplatform['Hulu'] == 1) & (cleaned_moviesonplatform['Netflix']== 0) & (cleaned_moviesonplatform['Prime Video'] == 0) & (cleaned_moviesonplatform['Disney+'] == 0)].index
cleaned_moviesonplatform.drop(Hulumovies,inplace=True)
final_moviesonplatform = cleaned_moviesonplatform.drop(['Hulu'], axis=1)
final_moviesonplatform.shape


(16105, 6)

In [ ]:
#Uploading TvShows File
tvshows = pd.read_csv("tv_shows.csv")
#Removing unnecessary columns to get in sync with moviesonplatform data
cleaned_tvshow = tvshows.drop (['ID', 'Unnamed: 0', 'Age', 'Rotten Tomatoes', 'IMDb'], axis = 1)
#Removing TV Shows present only on HULU
Hulu_shows = cleaned_tvshow[(cleaned_tvshow['Hulu'] == 1) & (cleaned_tvshow['Netflix']== 0) & (cleaned_tvshow['Prime Video'] == 0) & (cleaned_tvshow['Disney+'] == 0)].index
cleaned_tvshow.drop(Hulu_shows, inplace=True)
final_tvshow = cleaned_tvshow.drop(['Hulu'], axis=1)
final_tvshow.shape

(4034, 6)

In [ ]:
#Appending Moviesonstreaming and TVshows files
Finaldataset = final_moviesonplatform.append(final_tvshow)
Finaldataset.columns
Finaldataset.head()

,Title,Year,Netflix,Prime Video,Disney+,Type
0,Inception,2010,1,0,0,0
1,The Matrix,1999,1,0,0,0
2,Avengers: Infinity War,2018,1,0,0,0
3,Back to the Future,1985,1,0,0,0
4,"The Good, the Bad and the Ugly",1966,1,1,0,0


In [ ]:
#Renaming columns to get in sync with Netflix, Prime and Disney+ files
Finaldataset.rename(columns = {'Year':'release_year'}, inplace = True)
Finaldataset.rename(columns = {'Title':'title'}, inplace = True)
Finaldataset.rename(columns = {'Type':'type'}, inplace = True)
Finaldataset.head()

,title,release_year,Netflix,Prime Video,Disney+,type
0,Inception,2010,1,0,0,0
1,The Matrix,1999,1,0,0,0
2,Avengers: Infinity War,2018,1,0,0,0
3,Back to the Future,1985,1,0,0,0
4,"The Good, the Bad and the Ugly",1966,1,1,0,0


In [ ]:
#Appending all the 5 datasets together
onetimedata = Finaldataset.append(data_updated)
onetimedata.shape

(40064, 6)

In [ ]:
#Shuffling our data
f_final_dataset = onetimedata.sample(frac = 1)
f_final_dataset.reset_index(drop=True, inplace=True) 
f_final_dataset.shape

(40064, 6)

In [ ]:
#Dropping all the rows having same values in columns title, Netflix, Prime Video, Disney+, release_year and keeping last row as original
f_final_dataset.drop_duplicates(subset =['title', 'Netflix', 'Prime Video', 'Disney+', 'release_year'],
                     keep = 'last', inplace = True)
f_final_dataset.shape
df = pd.DataFrame(f_final_dataset)
df.to_csv('f_final_dataset.csv')

In [ ]:
'''duplicate_values = f_final_dataset[f_final_dataset.duplicated(subset='title')]
duplicate_values.shape'''

(2775, 6)

In [ ]:
'''x = f_final_dataset[f_final_dataset['title']=="Danny"]
df2 = x.groupby(['title', 'release_year']).sum()
df2'''

Netflix  Prime Video  Disney+  type
title   release_year                                     
What If 2013                0            1        0     0
        2014                0            1        0     0

In [ ]:
import requests
import json
x = f_final_dataset[0:500]
imdbratings =[]
country = []
genres =[]
language =[]
director =[]
cast =[]
censor_rating =[]
runtime = []
imdbvotes =[]
rotten_tomatoes = []
imdbid =[]
plot = []
totalseason = []
boxoffice = []
endyear =[]
for i in x['title']:
  web = 'https://www.omdbapi.com/?t='+i+'&apikey=9b925aaa'
  response_API = requests.get(web)
  h = response_API.text
  t = json.loads(h)
  if t['Response'] == 'True':
    imdbratings.append(t['imdbRating'])
    country.append(t['Country'])
    genres.append(t['Genre'])
    language.append(t['Language'])
    director.append(t['Director'])
    cast.append(t['Actors'])
    censor_rating.append(t['Rated'])
    runtime.append(t['Runtime'])
    imdbvotes.append(t['imdbVotes'])
    #rotten_tomatoes.append(t['Rotten Tomatoes'])
    imdbid.append(t['imdbID'])
    plot.append(t['Plot'])
    #totalseason.append(t['totalSeasons'])
    #boxoffice.append(t['BoxOffice'])
    endyear.append(t['Year'])
  else:
    imdbratings.append('0')
    cast.append('0')
    country.append('0')
    genres.append('0')
    language.append('0')
    director.append('0')
    censor_rating.append('0')
    runtime.append('0')
    imdbvotes.append('0')
    imdbid.append('0')
    plot.append('0')
    endyear.append('0')
    #totalseason.append('0')
    #boxoffice.append('0')
    continue
  

In [ ]:
x['imdbratings']=imdbratings
x['cast'] = cast
x['country'] = country
x['genres'] = genres
x['language'] = language
x['director'] = director
x['censor_rating'] = censor_rating
x['runtime'] = runtime
x['imdbVotes'] = imdbvotes
x['imdbid'] = imdbid
x['plot'] = plot
x['endyear'] = endyear

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
x

,title,release_year,Netflix,Prime Video,Disney+,type,imdbratings,cast,country,genres,language,director,censor_rating,runtime,imdbVotes,imdbid,plot,endyear
0,2019 Gold Series AVP New York City Open - Day 2,2019,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Junior,2013,0,1,0,0,4.7,"Arnold Schwarzenegger, Danny DeVito, Emma Thom...",United States,"Comedy, Romance, Sci-Fi",English,Ivan Reitman,PG-13,109 min,"67,838",tt0110216,"As part of a fertility research project, a mal...",1994
3,H.H. Holmes: America's First Serial Killer,2004,0,1,0,0,6.3,"Tony Jay, Harold Schechter, Thomas Cronin",United States,"Documentary, Crime, History",English,John Borowski,Unrated,64 min,"1,798",tt0358500,"Torture chambers, acid vats, greased chutes an...",2004
4,Christopher Robin,2018,0,0,1,0,7.3,"Ewan McGregor, Hayley Atwell, Bronte Carmichael","United Kingdom, United States","Animation, Adventure, Comedy",English,Marc Forster,PG,104 min,"74,126",tt4575576,"A working-class family man, Christopher Robin,...",2018
5,Grego Rossello: Disculpe las molestias,2019,1,0,0,0,4.5,Grego Rossello,Argentina,Comedy,Spanish,Juani Libonatti,TV-MA,65 min,25,tt11066396,Gregorio Rosello is a stand up comedian. In hi...,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,Vials,2017,0,1,0,1,7.3,"Sanjay Rao, Elena Weinberg, David Lampe",United States,Comedy,English,N/A,N/A,30 min,279,tt6992418,"The series follows the lives of pharmacists, i...",2017–
654,Night Feeder,2021,0,1,0,0,5.9,"Kate Alexander, The Nuns, Cintra Wilson, Jonat...",USA,Horror,English,Jim Whiteaker,N/A,90 min,69,tt4777164,Fear chokes the free-wheeling underbelly of Sa...,1988
655,Hollywood Cop,1987,0,1,0,0,4.1,"James Mitchum, Cameron Mitchell, David Goss",United States,"Action, Crime, Drama",English,Amir Shervan,R,101 min,522,tt0095313,A detective tries to get back a child who has ...,1987
656,Aurinko In Adagio: A Rising Voices Film,2021,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = pd.DataFrame(x)
df.to_csv('final_dataset.csv')

from pandas_profiling import ProfileReport

In [ ]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
X

SyntaxError: ignored

In [ ]:
pip install 'https://github.com/pandas-profiling/pandas-profiling/archive/master.zip'

     \ 25.9 MB 181 kB/s
     |████████████████████████████████| 10.9 MB 7.7 MB/s 
     |████████████████████████████████| 596 kB 82.1 MB/s 
     |████████████████████████████████| 102 kB 12.9 MB/s 
     |████████████████████████████████| 675 kB 68.2 MB/s 
     |████████████████████████████████| 4.7 MB 46.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 812 kB 47.6 MB/s 
     |████████████████████████████████| 38.1 MB 341 kB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.1.1-py2.py3-none-any.whl size=261287 sha256=917eb1c4d30ad54f95e054840ac122b4d5749ee66cd439ae25e39d2bfb1f7a68
  Stored in directory: /tmp/pip-ephem-wheel-cache-974ckbb1/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=4b336dbb7365ef0b48b6e56235cc5544fa1a7abc58ed527ee5ade3b5522ed857
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14

In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport
x = pd.read_csv('final_dataset.csv')
profile = ProfileReport(x, title="Pandas Profiling Report")

In [ ]:
profile.to_file("profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
pip install sweetviz

     |████████████████████████████████| 15.1 MB 5.1 MB/s 


In [ ]:
import pandas as pd
import sweetviz as sv
df = pd.read_csv('final_dataset.csv')
df.head()

,Unnamed: 0,title,release_year,Netflix,Prime Video,Disney+,type,imdbratings,cast,country,genres,language,director,censor_rating,runtime,imdbVotes,imdbid,plot,endyear
0,0,2019 Gold Series AVP New York City Open - Day 2,2019,0,1,0,1,0.0,0,0,0,0,0,0,0,0,0,0,0
1,2,Junior,2013,0,1,0,0,4.7,"Arnold Schwarzenegger, Danny DeVito, Emma Thom...",United States,"Comedy, Romance, Sci-Fi",English,Ivan Reitman,PG-13,109 min,"67,838",tt0110216,"As part of a fertility research project, a mal...",1994
2,3,H.H. Holmes: America's First Serial Killer,2004,0,1,0,0,6.3,"Tony Jay, Harold Schechter, Thomas Cronin",United States,"Documentary, Crime, History",English,John Borowski,Unrated,64 min,"1,798",tt0358500,"Torture chambers, acid vats, greased chutes an...",2004
3,4,Christopher Robin,2018,0,0,1,0,7.3,"Ewan McGregor, Hayley Atwell, Bronte Carmichael","United Kingdom, United States","Animation, Adventure, Comedy",English,Marc Forster,PG,104 min,"74,126",tt4575576,"A working-class family man, Christopher Robin,...",2018
4,5,Grego Rossello: Disculpe las molestias,2019,1,0,0,0,4.5,Grego Rossello,Argentina,Comedy,Spanish,Juani Libonatti,TV-MA,65 min,25,tt11066396,Gregorio Rosello is a stand up comedian. In hi...,2019


In [ ]:
my_report = sv.analyze(df)
my_report.show_html() 

                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
